# Import Libraries

In [119]:
# data manipulation
import pandas as pd

# numpy arrays
import numpy as np

# data visualization
import seaborn as sns

import matplotlib.pyplot as plt

import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

sns.set()

# NLP
import string

from wordcloud import WordCloud

import nltk
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

from NLTKVectorizer import NLTKVectorizer

import re

# machine learning
from sklearn.datasets import fetch_20newsgroups

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression  # Logistic Regression
from sklearn.naive_bayes import MultinomialNB  # Naive Bayes
from sklearn.svm import LinearSVC  # SVM
from sklearn.ensemble import RandomForestClassifier  # Random Forest

from sklearn.decomposition import TruncatedSVD

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.calibration import CalibratedClassifierCV

# Model explainability
from lime.lime_text import LimeTextExplainer

# other
from pprint import pprint
from time import time
import logging
from functools import partial
import joblib

import nltk
nltk.download('punkt')

import nltk
nltk.download('stopwords')

en_stop_words = list(set(stopwords.words("english")))

# Helper Functions

In [120]:
dfTrain = pd.read_csv('Dataset/archive/train.csv')
dfTest = pd.read_csv('Dataset/archive/test.csv')

In [121]:
X_train = dfTrain['question_title']
y_train = dfTrain['topic']
X_test = dfTest['question_title']
y_test = dfTest['topic']

KeyboardInterrupt: 

## Color palette generator

This function is used to create a *palette* of `n` colors of `palette_name` colors.

In [ ]:
def get_n_color_palette(palette_name, n_colors, as_hex=False):
    palette = sns.color_palette(palette=palette_name, n_colors=n_colors)
    if as_hex:
        palette = palette.as_hex()
    palette.reverse()
    return palette

## Plotly export chart

This function is used to export the HTML of plotly `fig_obj`, and save it in: `assets/file_name.html`

In [ ]:
def save_fig_as_div(fig_obj, file_name):
    with open(f"assets/{file_name}", "w") as fig_file:
        fig_div_string = plotly.offline.plot(
            figure_or_data=fig_obj, output_type="div", include_plotlyjs="cdn"
        )
        fig_file.write(fig_div_string)

## Classification report

This function generates the classification report for the model predictions

In [ ]:
def get_classification_report(y_true, y_pred, target_names):

    # calculate classification report, and convert it to DataFrame
    clf_report = classification_report(
        y_true=y_true, y_pred=y_pred, target_names=target_names, output_dict=True
    )
    clf_report_df = pd.DataFrame(data=clf_report)
    clf_report_df = clf_report_df.T
    clf_report_df.drop(columns=["support"], inplace=True)

    measures = clf_report_df.columns.tolist()
    classes = clf_report_df.index.tolist()

    # create plotly annotated heatmap, and update styling
    fig = ff.create_annotated_heatmap(clf_report_df.values, x=measures, y=classes)
    fig.update_layout(
        autosize=False,
        width=800,
        height=800,
        title_text="<i><b>Classification report</b></i>",
        xaxis_title="Measures",
        yaxis_title="Class",
        plot_bgcolor="rgba(0, 0, 0, 0)",
        paper_bgcolor="rgba(0, 0, 0, 0)",
        font={
            "family": "Courier New, monospace",
            "size": 14,
            # 'color': "#eaeaea"
        },
    )
    fig.update_xaxes(tickangle=-45)
    fig["data"][0]["showscale"] = True

    return fig

## Confusion matrix

This function generates the confusion matrix for the model predictions

In [ ]:
def get_confusion_matrix(y_true, y_pred, labels):

    # claculate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=labels)
    conf_matrix = np.flipud(conf_matrix)

    # create annotated heat map of the confusion matrix
    fig = ff.create_annotated_heatmap(
        conf_matrix, x=labels.tolist(), y=labels.tolist()[::-1]
    )
    fig.update_layout(
        autosize=False,
        width=800,
        height=800,
        title_text="<i><b>Confusion matrix</b></i>",
        xaxis_title="Predicted category",
        yaxis_title="Real category",
        plot_bgcolor="rgba(0, 0, 0, 0)",
        paper_bgcolor="rgba(0, 0, 0, 0)",
        font={
            "family": "Courier New, monospace",
            "size": 14,
            # 'color': "#eaeaea"
        },
    )
    fig.update_xaxes(tickangle=-45)
    fig["data"][0]["showscale"] = True

    return fig

# Load and Prepare Data

In [ ]:
newsgroups_data = fetch_20newsgroups(subset="all")

In [ ]:
categories = newsgroups_data.target_names

In [ ]:
df = pd.DataFrame(
    data={"text": newsgroups_data.data, "category": newsgroups_data.target}
)

In [ ]:
df["category"] = df["category"].apply(lambda x: categories[x])

In [ ]:
df.head()

In [ ]:
df.shape

The dataset consists of 18,846 samples

# What is this Data?

The 20 Newsgroups data set is a collection of approximately 20,000 newsgroup documents, partitioned (nearly) evenly across 20 different newsgroups.

The data is organized into 20 different newsgroups, each corresponding to a different topic. Some of the newsgroups are very closely related to each other (e.g. `comp.sys.ibm.pc.hardware` / `comp.sys.mac.hardware`), while others are highly unrelated (e.g `misc.forsale` / `soc.religion.christian`).

Here is a list of the 20 newsgroups, partitioned (more or less) according to subject matter:

<table style='font-family:"Courier New", Courier, monospace; font-size:80%'>
    <tr>
        <td>comp.graphics<br>comp.os.ms-windows.misc<br>comp.sys.ibm.pc.hardware<br>comp.sys.mac.hardware<br>comp.windows.x
        </td>
        <td>rec.autos<br>rec.motorcycles<br>rec.sport.baseball<br>rec.sport.hockey</td>
        <td>sci.crypt<br>sci.electronics<br>sci.med<br>sci.space</td>
    </tr>
    <tr>
        <td>misc.forsale</td>
        <td>talk.politics.misc<br>talk.politics.guns<br>talk.politics.mideast</td>
        <td>talk.religion.misc<br>alt.atheism<br>soc.religion.christian</td>
    </tr>
</table>

# Data Statistics

One quick *data-cleaning* step is to rename the category columns to something more readable:

In [ ]:
df = df.assign(category=df["category"].apply(lambda x: "_".join(x.split(".")[1:])))

In [ ]:
df["category"].unique()

Calculate for each topic in the dataset the number of topics

In [ ]:
categories_statistics_df = (
    dfTrain.groupby(by="topic")["id"]
    .agg(
        [
            ("count", lambda x: x.size),
        ]
    )
    .reset_index()
    .sort_values(by="count", ascending=False)
)

Calculate for each topic in the dataset the average length of answers

In [ ]:
categories_statistics_df_questions = (
    dfTrain.groupby(by="topic")["question_title"]
    .agg(
        [
            ("mean", lambda x: x.str.len().mean()),
            ("max", lambda x: x.str.len().max()),
            ("min", lambda x: x.str.len().min()),
        ]
    )
    .reset_index()
)

## Categories article count:

Use a pie chart to the percentages of articles for each category:

In [ ]:
blue_palette = get_n_color_palette("Blues", 20, True)

fig = px.pie(
    data_frame=categories_statistics_df,
    names="topic",
    values="count",
    color_discrete_sequence=blue_palette,
    title="Categories Percentages",
    width=800,
    height=500,
)

fig.update_layout(
    {
        "plot_bgcolor": "rgba(0, 0, 0, 0)",
        "paper_bgcolor": "rgba(0, 0, 0, 0)",
        "font": {
            "family": "Courier New, monospace",
            "size": 14,
            # 'color': "#eaeaea"
        },
    }
)

fig.show()

We can see that the dataset is *balanced*

In [ ]:
save_fig_as_div(fig, file_name="charts/categories-percentages-pie-chart.html")

## Categories average article length:

Use a bar chart to show the average article length per category:

In [ ]:
chart_labels = {"mean": "Question title Length", "Topic": "Topic type"}

fig = px.bar(
    data_frame=categories_statistics_df_questions.sort_values(by="mean"),
    x="topic",
    y="mean",
    color="mean",
    labels=chart_labels,
    title="Average Article Length by Category",
    width=800,
    height=500,
)

fig.update_layout(
    {
        "plot_bgcolor": "rgba(0, 0, 0, 0)",
        "paper_bgcolor": "rgba(0, 0, 0, 0)",
        "font": {
            "family": "Courier New, monospace",
            "size": 14,
            # 'color': "#eaeaea"
        },
    }
)

# rotate x-axis ticks
fig.update_xaxes(tickangle=-45)

fig.show()

This chart shows how lengthy the *polictics* articles, compared to *computer* articles

In [ ]:
save_fig_as_div(fig, file_name="charts/average-article-length-bar-chart.html")

# Word Cloud

In [ ]:
categories_text_df = dfTrain.groupby(by="topic").agg({"question_title": " ".join}).reset_index()

In [ ]:
def plot_word_cloud(category_name, category_text):
    plt.subplots(figsize=(8, 8))
    wc = WordCloud(
        background_color="white", stopwords=en_stop_words, width=1000, height=600
    )
    wc.generate(category_text)
    plt.title(label=category_name)
    plt.axis("off")
    plt.imshow(wc, interpolation="bilinear")
    plt.show()

The following word-clouds will help us to take a glimpse on the data, and its content.

Each word cloud shows the relative frequency of words in a category, words with higher *frequencies* have bigger *size*.

This will help us to see what are the **dominant** words in each category.

In [ ]:
for idx, row in categories_text_df.iterrows():
    plot_word_cloud(row["topic"], row["question_title"])

# Text Vectorization

In this step, we'll build a **text vectorization** transformer which will be used to convert the raw text documents into appropriate features, prepared to be input for machine learning algorithms.

I will define a *custom vectorizer* called `NLTKVectorizer`. This vectorizer inherits the `TfidfVectorizer` and overrides the `build_analyzer` method. The resulting vectorizer will have the same parameters as the `TfidfVectorizer` but it will *analyze* the documents in a different manner, mainly it will use `NLTK` tokenzier, lemmatizer.

# Create Pipeline

Use a very minimalistic Pipeline consisting of only two stpes:
- Text Vectorizer (Transformer): in this step the vectorizer takes the raw text input, perform some data cleaning, text representation (using TF-IDF), and returns an array of features for each sample in the dataset.
- Classifier (estimator): the classifier then takes the output produced by the previous step (which is features matrix), and use it as input to traing machine learning algorithm to learn from the data.

Why pipelines?

Pieplines are the recommended approach of combining data processing steps with machine learning stpes, they let use *seamlessly* apply many transformations on the input data, and finally train a model on the produced data.

They are considered as one of the best-practises in `sklearn`.

Here, we are creating a two-step pipeline, which may doesn't show how important it is, but it's very common to create a very complex pipline consisting of many data transformations steps followed by a machine learning model.

Another bonus of pieplines, is that they can help us perform cross validation on the data all at once, and tune the hyper-parameters.

There are many great resources about the importance of using pipelines, and how to use them:

- [Deploying Machine Learning using sklearn pipelines](https://www.youtube.com/watch?v=URdnFlZnlaE)
- [A Simple Example of Pipeline in Machine Learning with Scikit-learn](https://towardsdatascience.com/a-simple-example-of-pipeline-in-machine-learning-with-scikit-learn-e726ffbb6976)
- [A Deep Dive Into Sklearn Pipelines](https://www.kaggle.com/baghern/a-deep-dive-into-sklearn-pipelines)

In [ ]:
# text vectorizer
vectorizer = NLTKVectorizer(
    stop_words=en_stop_words, max_df=0.5, min_df=10, max_features=10000
)

# Logistic Regression classifier
lr_clf = LogisticRegression(C=1.0, solver="newton-cg", multi_class="multinomial")

# Naive Bayes classifier
nb_clf = MultinomialNB(alpha=0.01)

# SVM classifier
svm_clf = LinearSVC(C=1.0)

# Random Forest classifier
random_forest_clf = RandomForestClassifier(
    n_estimators=100, criterion="gini", max_depth=50, random_state=0
)

In [ ]:
pipeline = Pipeline([("vect", vectorizer), ("clf", lr_clf)])

# Hyper-parameters tunning

**NOTE**: only perform the `GridSearch` when you want to find the best parameters, once the best parameters are found, use them to create the best piepline.

Choosing the right parameters of our text vectorizer and classifier is a troublesome, there are many parameters, we need to assign a value for each parameter, train the model, and see the accuracy of our model.

This trail-and-error approach could be summarized as the following:
- select one parameter of the hyper-parameters at a time.
- assign this parameter an appropriate value (depending on this parameter represent).
- train the model.
- compare the accuracy
- repeat for another parameter.

Luckily, `sklearn` supports tunning the hyper-parameters in a much nicer way using its `GridSearchCV` class, which performs **Exhaustive** search over the specified parameters settings, and beside searching for the best parameters, it optimize the results by cross validating the model, and reporting the average model across the splits.

`sklearn` supports another approach for tuning the hyper-parameters, they are explaind in details in the [documentation](https://scikit-learn.org/stable/modules/grid_search.html) 

The parameter settings to tune:

In [ ]:
# parameters = {
#     # vectorizer hyper-parameters
#     'vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
#     'vect__max_df': [0.4, 0.5, 0.6],
#     'vect__min_df': [10, 50, 100],
#     'vect__max_features': [5000, 10000],
#     # classifiers
#     'clf': [lr_clf, nb_clf, svm_clf, random_forest_clf]
# }

In [ ]:
# grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, refit=True)

# print("Performing grid search...")
# print("pipeline:", [name for name, _ in pipeline.steps])
# print("parameters:")
# pprint(parameters)
# t0 = time()
# grid_search.fit(X_train, y_train)
# print("done in %0.3fs\n" % (time() - t0))

# print("Best score: %0.3f" % grid_search.best_score_)
# print("Best parameters set:")
# best_parameters = grid_search.best_estimator_.get_params()
# for param_name in sorted(parameters.keys()):
#     print("\t%s: %r" % (param_name, best_parameters[param_name]))

## What are the best vectorizer and classifier params:

In [ ]:
# grid_search.best_params_

Pipeline best params:

{'clf': LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
       intercept_scaling=1, loss='squared_hinge', max_iter=1000,
       multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
       verbose=0),
'vect__max_df': 0.5,
'vect__max_features': 10000,
'vect__min_df': 10,
'vect__ngram_range': (1, 1)}

the best parameters reported after performing grid search:
- `clf`: the classifier which achieved the highest score is the `LinearSVC`.
- `max_df`: terms that have a document frequency higher than 50% are ignored.
- `min_df`: terms that have document frequency strictly lower than 10 are ignored.
- `ngram_range`: using only unigrams.
- `max_features`: selecting only 10,000 features.

## What is the best estimator:

In [ ]:
# grid_search.best_estimator_

# Best classifier

Create a pipeline with the best parameters found by the grid search:

In [ ]:
vectorizer = NLTKVectorizer(
    max_df=0.5,
    min_df=10,
    ngram_range=(1, 1),
    max_features=10000,
    stop_words=en_stop_words,
)

svm_clf = LinearSVC(C=1.0)

The `LinearSVC` class, has only `predict` method, while other classifiers has the `predict_proba` method. Later on we'll need the `predict_proba` mthod for predicting probabilities for samples, to use it in explaining the model predictions.

One way to obtain the probabilities of the classifier is to wrap it in `CalibratedClassifierCV` class, after using this class we'll be able to use the `predict_proba` method.

Resources about how the `CalibratedClassifierCV` works, and how to use it:

- [Sklearn Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.calibration.CalibratedClassifierCV.html)
- [Converting LinearSVC's decision function to probabilities (Scikit learn python)](https://stackoverflow.com/questions/26478000/converting-linearsvcs-decision-function-to-probabilities-scikit-learn-python/39712590#39712590)
- [Scikit correct way to calibrate classifiers with CalibratedClassifierCV](https://stats.stackexchange.com/questions/263393/scikit-correct-way-to-calibrate-classifiers-with-calibratedclassifiercv/263411#263411)

In [ ]:
clf = CalibratedClassifierCV(base_estimator=svm_clf, cv=5, method="isotonic")

In [ ]:
pipeline = Pipeline([("vect", vectorizer), ("clf", clf)])

Fit the pipeline on the training data, and then evaluate it on the test data.

In [ ]:
# text vectorizer
vectorizer = NLTKVectorizer(stop_words=en_stop_words,
                            max_df=0.5, min_df=10, max_features=10000)

# logistic Regression classifier
lr_clf = LogisticRegression(C=1.0, solver='newton-cg', multi_class='multinomial', n_jobs=-1, verbose=2)

# create pipeline object
pipeline = Pipeline([
    ('vect', vectorizer),
    ('clf', lr_clf)
])


In [ ]:
nltk.download('punkt')

In [ ]:
nltk.download('wordnet')

In [ ]:
%%time
# = RandomForestRegressor(n_estimators=100, verbose=2, n_jobs=2).fit(X_train, y_train)
model = pipeline.fit(X_train, y_train)


In [ ]:
%%time
y_pred = pipeline.predict(X_test)

# Model Evaluation

## Classification report:

In [ ]:
fig = get_classification_report(
    y_true=y_test, y_pred=y_pred, target_names=pipeline.classes_
)
fig.show()

In [ ]:
save_fig_as_div(fig_obj=fig, file_name="charts/classification-report.html")

## Confusion matrix:

In [ ]:
fig = get_confusion_matrix(y_true=y_test, y_pred=y_pred, labels=pipeline.classes_)
fig.show()

In [ ]:
save_fig_as_div(fig_obj=fig, file_name="charts/confusion-matrix.html")

# Model Explainability

At this point we should be done, right? after training the model and evaluating it, and achieving good results we should stop.

There are many unanswered questions so far, for example:
- How the model is working and making predictions (what are the features).
- When the model predicts coorectly and when it's not.
- Does the model generalize on the data.
- Debugging the model on certain predictions.

And most importantly, is this model reliable? can we use it in production with confidence?

We can think of our model more or less like a *Black Box*, it takes some input, and produce some output.

![alt text](assets/balck-box-ml.png "Black Box Machine Learning")

The field of *Explainable artificial intelligence* which is concerned with the tools and methods for explaining and interpreting machine learning algorithms, catched a large interest in the past few years, and there has been many libraries that can be used out of the box for interpreting machine learning and deep learning models:
- [eli5 (short for: Explain like I'm 5)](https://github.com/TeamHG-Memex/eli5)
- [Lime: Explaining the predictions of any machine learning classifier](https://github.com/marcotcr/lime)
- [SHAP: A game theoretic approach to explain the output of any machine learning model](https://github.com/slundberg/shap)

These libraries differ in the way they work, and the type of models they can *interpret*

## Model complexity vs interpretability:

*Simple* linear models are easy to explain (since they consists of only linear equations), but their accuracy is low compared to *Complex* non-linear models, which achieve higher accuracy, but are harder to explain.

The following figure illustrates the relation between model accuracy and interpretability (Source: [The balance: Accuracy vs. Interpretability](https://towardsdatascience.com/the-balance-accuracy-vs-interpretability-1b3861408062)):

![alt text](assets/accuracy-vs-interpretability.png "Accuracy vs Interpretability")

Since we're using a *linear SVM* classifier, we'll try to visualize the weights assigned to the features, and see for each class (category) in our data, what are the features that affect the model's prediction positivly and negatively.

Resources about interpreting SVM classifier in `sklearn`:
- [How does one interpret SVM feature weights?](https://stats.stackexchange.com/questions/39243/how-does-one-interpret-svm-feature-weights/39311#39311)
- [Visualising Top Features in Linear SVM with Scikit Learn and Matplotlib](https://medium.com/@aneesha/visualising-top-features-in-linear-svm-with-scikit-learn-and-matplotlib-3454ab18a14d)

## Visualizing classifier weights:

In [ ]:
feature_names = pipeline["vect"].get_feature_names()

In [ ]:
clf = pipeline.named_steps["clf"]

In [ ]:
coefs_values = sum(
    [classifier.base_estimator.coef_ for classifier in clf.calibrated_classifiers_]
)

coefs_values = coefs_values / len(clf.calibrated_classifiers_)

In [ ]:
classes = clf.classes_

In [ ]:
n_feature = 15

In [ ]:
for i, class_label in enumerate(classes):

    # get indices of top positive/negative coefficient
    negative_coefs_indices = np.argsort(coefs_values[i])[:n_feature]
    positive_coefs_indices = np.argsort(coefs_values[i])[-n_feature:]

    # get the coefficient values
    negative_coefs = [coefs_values[i][coef_idx] for coef_idx in negative_coefs_indices]
    positive_coefs = [coefs_values[i][coef_idx] for coef_idx in positive_coefs_indices]

    # get the corresponding features names of the top coefficient
    negative_features = [feature_names[coef_idx] for coef_idx in negative_coefs_indices]
    positive_features = [feature_names[coef_idx] for coef_idx in positive_coefs_indices]

    # stack arrays into one array
    weights = np.concatenate([negative_coefs, positive_coefs])
    features = np.concatenate([negative_features, positive_features])

    # plot feature names agains their weight, using bar plot
    chart_title = f"{class_label} articles"

    chart_labels = {"x": "Feature name", "y": "Feature weight"}

    fig = px.bar(
        x=features,
        y=weights,
        color=weights,
        title=chart_title,
        orientation="v",
        labels=chart_labels,
        width=800,
        height=500,
        color_continuous_scale=["red", "blue"],
    )

    # rotate x-axis ticks
    fig.update_xaxes(tickangle=45)

    fig.update_layout(
        {
            "plot_bgcolor": "rgba(0, 0, 0, 0)",
            "paper_bgcolor": "rgba(0, 0, 0, 0)",
            "font": {
                "family": "Courier New, monospace",
                "size": 14,
                # 'color': "#eaeaea"
            },
        }
    )

    fig.show()

    save_fig_as_div(
        fig, file_name=f"model-coefficients/{class_label}-class-bar-chart.html"
    )

## Explaining individual predictions:

In [ ]:
clf_predictions_df = pd.DataFrame(
    data={
        "text": X_test.values,
        "real category": y_test.values,
        "predicted category": y_pred,
    }
)

clf_predictions_df["text length"] = clf_predictions_df["text"].str.len()

clf_predictions_df.sort_values(by="text length", ascending=False, inplace=True)

In [ ]:
clf_predictions_df.head()

In [ ]:
# samples which has been correctly classified
correct_classified_df = clf_predictions_df[
    clf_predictions_df["real category"] == clf_predictions_df["predicted category"]
]

In [ ]:
# samples which has been incorrectly classified
incorrect_classified_df = clf_predictions_df[
    clf_predictions_df["real category"] != clf_predictions_df["predicted category"]
]

In [ ]:
# preprocessing function used in the pipeline to analyze input documents
tokenize_fn = pipeline.named_steps["vect"].build_analyzer()

In [ ]:
explainer = LimeTextExplainer(verbose=True, class_names=pipeline.classes_)

In [ ]:
clf_fn = pipeline.predict_proba

### When the model is performing well:

In [ ]:
correct_sample = correct_classified_df.loc[[2573]]

In [ ]:
correct_sample

In [ ]:
text_to_explain = correct_sample["text"].values[0]
cleaned_text_to_explain = " ".join(tokenize_fn(text_to_explain))

In [ ]:
exp_object = explainer.explain_instance(
    text_instance=cleaned_text_to_explain,
    classifier_fn=clf_fn,
    top_labels=2,
    num_features=10,
    num_samples=10000,
)

In [ ]:
labels = exp_object.available_labels()

In [ ]:
exp_object.show_in_notebook(labels=labels)

In [ ]:
exp_object.save_to_file(
    file_path="assets/model-explanations/correct-classification-explanation.html",
    labels=labels,
)

Here the classifier predicted the correct class of the sample

In [ ]:
# politics_mideast class
exp_object.as_list(label=10)

We can see that words like `omar` and `hand` are negative fearues for this class, and if we remove them from the document, the the prediction percentage of class `politics_mideast` would increase.

In [ ]:
cleaned_text_to_explain = re.sub("omar|hand", "", cleaned_text_to_explain)

In [ ]:
exp_object = explainer.explain_instance(
    text_instance=cleaned_text_to_explain,
    classifier_fn=clf_fn,
    top_labels=2,
    num_features=10,
    num_samples=10000,
)

In [ ]:
labels = exp_object.available_labels()

In [ ]:
exp_object.show_in_notebook(labels=labels)

### When the model is performing badly:

In [ ]:
incorrect_sample = incorrect_classified_df.loc[[3145]]

In [ ]:
incorrect_sample

In [ ]:
text_to_explain = incorrect_sample["text"].values[0]
cleaned_text_to_explain = " ".join(tokenize_fn(text_to_explain))

In [ ]:
exp_object = explainer.explain_instance(
    text_instance=cleaned_text_to_explain,
    classifier_fn=clf_fn,
    top_labels=2,
    num_features=10,
    num_samples=10000,
)

In [ ]:
labels = exp_object.available_labels()

In [ ]:
exp_object.show_in_notebook(labels=labels)

Here the classifier predicted an incorrect class, it predicted the class `autos` while the correct class is `forsale`, the reason is that the document very short, it contains two words about selling, and two words about cars, and the classification probabilities for the two classes were almost equal, let's try another example:

In [ ]:
incorrect_sample = incorrect_classified_df.loc[[4251]]

In [ ]:
incorrect_sample

Here, the classifier had done the opposite of the previous case, it predicted the class `forsale` while the real class is `autos`.

In [ ]:
text_to_explain = incorrect_sample["text"].values[0]
cleaned_text_to_explain = " ".join(tokenize_fn(text_to_explain))

In [ ]:
exp_object = explainer.explain_instance(
    text_instance=cleaned_text_to_explain,
    classifier_fn=clf_fn,
    top_labels=2,
    num_features=10,
    num_samples=10000,
)

In [ ]:
labels = exp_object.available_labels()

In [ ]:
exp_object.show_in_notebook(labels=labels)

What happened here is that the classifier had correlated words like `seat` and `owner` with class `autos`, and words like `sale`, `original` and `sell` with class `forsale`, we can inspect the sample more to understand why it's so confusing:

In [ ]:
print(text_to_explain)

Reading through the article, I *myself* actually got confused! so the article goes about someone criticizing the idea of posting car-seats ads, and he mentions two car models, the [Toyota MR2](https://en.wikipedia.org/wiki/Toyota_MR2) and the [Toyota Celica](https://en.wikipedia.org/wiki/Toyota_Celica), these two words are very informative to recognize the `autos` class, but it's hard for the classifier to catch them (since it's based on frequency statistics).

In [ ]:
exp_object.save_to_file(
    file_path="assets/model-explanations/incorrect-classification-explanation.html",
    labels=labels,
)

# Improving accuracy

## Edit normalization

After inspecting how the model is predicting classes, we saw that word `MR2` is actually relevant to the classification, but it's being removed by the analyzer, I'll change the value of `min_token_length` to 2, this way only tokens of two letters or less will be removed.

In [ ]:
vectorizer = NLTKVectorizer(
    max_df=0.5,
    min_df=10,
    ngram_range=(1, 1),
    max_features=10000,
    stop_words=en_stop_words,
    min_token_length=2,
)

svm_clf = LinearSVC(C=1.0)

In [ ]:
clf = CalibratedClassifierCV(base_estimator=svm_clf, cv=5, method="isotonic")

In [ ]:
pipeline = Pipeline([("vect", vectorizer), ("clf", clf)])

In [ ]:
%%time
pipeline.fit(X_train, y_train)

In [ ]:
%%time
y_pred = pipeline.predict(X_test)

## Classification report

In [ ]:
fig = get_classification_report(
    y_true=y_test, y_pred=y_pred, target_names=pipeline.classes_
)
fig.show()

In [ ]:
save_fig_as_div(fig_obj=fig, file_name="charts/classification-report-improved.html")

## Confusion matrix

In [ ]:
fig = get_confusion_matrix(y_true=y_test, y_pred=y_pred, labels=pipeline.classes_)
fig.show()

In [ ]:
save_fig_as_div(fig_obj=fig, file_name="charts/confusion-matrix-improved.html")

## AI model Dump

In [ ]:
from joblib import dump, load

In [ ]:
dump(pipeline, 'dump/test.joblib')

## Load AI model and use it

In [ ]:
pipeline = load('dump/test.joblib')

In [161]:
text = input('Enter your question: ')

predictProba = pipeline.predict_proba([text])

array = ["Society & culture", "Science & Mathematics", "Health", "Education & Reference", "Computers & Internet", "Sports", "Business & Finance", "Entertainment & Music", "Family & Relationships", "Politics & Government"]
stat = predictProba[0][pipeline.predict([text])[0]] * 100

print("\"" + text + "\"" + "\n" " is about: " + "\"" + array[pipeline.predict([text])[0]]+ "\"" +"\n"+" with a probability of " +"%.3f" %stat + "%" "\n")

for i in range(10):
    print("\t\t\t"+ array[i] + " : " + "%.3f" % (predictProba[0][i] * 100) + "%")

"suca"
 is about: "Science & Mathematics"
 with a probability of 12.629%

			Society & culture : 10.745%
			Science & Mathematics : 12.629%
			Health : 10.282%
			Education & Reference : 12.197%
			Computers & Internet : 6.159%
			Sports : 8.081%
			Business & Finance : 11.313%
			Entertainment & Music : 11.280%
			Family & Relationships : 11.130%
			Politics & Government : 6.183%
